In [1]:
import os
import geopandas as gpd
import numpy as np
from tqdm import tqdm

In [2]:
import matplotlib.pyplot as plt

In [3]:
MERGED_DATA_PATH = '../Data/Speed Data/Merged' 
GRADIENT_DATA_PATH = '../Data/Gradient Data/Adjusted/'
OUTPUT_PATH = '../Data/Cycle Data/Original'

In [4]:
def closest(geom,possible):
    '''
    
    Input:
    geom - LineString geometry of bus route segment
    possible - LineString geometry of associated speed mesh segments found through spatial joining
    
    Output:
    i - index of possible which has the smallest angle with geom
    
    '''
    dy = geom.coords[1][1] - geom.coords[0][1]
    dx = geom.coords[1][0] - geom.coords[0][0]
    
    theta = np.arctan2(dy,dx)
    
    ans = 0
    close = 100000
    
    for i,e in enumerate(possible):
        dy = e.coords[1][1] - e.coords[0][1]
        dx = e.coords[1][0] - e.coords[0][0]
    
        t_theta = np.arctan2(dy,dx)
        
        if(abs(theta-t_theta) < close):
            close = abs(theta-t_theta)
            ans = i
    
    return i

In [5]:
def add_speed(df, route):
    
    '''
    Uses spatial joining to add speed data to route GeoDataFrame
    
    Input:
    df - GeoDataFrame of bucketed speed mesh
    route - GeoDataFrame of bus route 
    
    Output
    None
    '''
    result = gpd.sjoin(route,df)
    
    speeds = np.zeros(len(route['geometry']))
            
    for idx in set(result.index):
        cur = list(result['index_right'][result.index == idx])

        ind = closest(route['geometry'][idx],list(df['non_buffered'][cur]))
        poss = list(df['speed'][cur])

        speeds[idx] = poss[ind]

    route['speed'] = speeds
    

In [6]:
def to_cycle(route, out_path):
    '''
    TODO: DOCUMENTATION
    '''
    speeds = []
    grades = []

    time_left = 1
    cur_speed = 0
    cur_grade = 0
    empty = 0

    avg_speed = np.mean(route['speed'][route['speed'] != 0])
    
    for i in range(len(route['geometry'])):
        geom = route['geometry'][i]
        speed = route['speed'][i] 
        if speed == 0:
            empty += 1
            speed = avg_speed
        grade = route['grade'][i]
        len_left = geom.length
        while(len_left > 0):
            if(len_left > speed*time_left):
                len_left -= speed*time_left
                cur_speed += time_left*speed
                cur_grade += time_left*grade
                grades.append(cur_grade)
                speeds.append(cur_speed)
                cur_speed = 0
                cur_grade = 0
                time_left = 1
            else:
                time_spent = len_left/speed 
                len_left = 0
                time_left -= time_spent
                cur_speed += time_spent*speed
                cur_grade += time_spent*grade
                
    f = open(out_path, "w")
    f.write("cycSecs,cycMps,cycGrade,cycRoadType\n")
    for i in range(len(grades)):
        f.write(str(i) + "," + str(speeds[i]) + "," + str(grades[i]) +",0\n")
    f.close()

In [7]:
speed_dfs = []


for month_bucket in range(4):
    speed_dfs.append([])
    
for k in tqdm(range(12),'Loading Speed Data'):
    month_bucket, hour_bucket = k//3, k%3
    path = os.path.join(MERGED_DATA_PATH, "%d_%d.shp" % (month_bucket, hour_bucket))
    df = gpd.read_file(path)
    df = df.to_crs('epsg:32614')
    
    df['non_buffered'] = df['geometry']
    df['geometry'] = df['geometry'].buffer(20)

    speed_dfs[month_bucket].append(df)

Loading Speed Data:   8%|▊        | 1/12 [00:20<03:46, 20.59s/it]


KeyboardInterrupt: 

In [8]:
for month_bucket in range(4):
    for hour_bucket in range(3):
        bucket = "%d_%d" % (month_bucket, hour_bucket)
        bucket_path = os.path.join(OUTPUT_PATH, bucket)
        os.mkdir(bucket_path)

FileExistsError: [WinError 183] Cannot create a file when that file already exists: '../Data/Cycle Data/Original\\0_0'

In [9]:
shps = []
for file in os.listdir(GRADIENT_DATA_PATH):
    if ".shp" in file:
        shps.append(file)

for file in tqdm(shps, 'Cycle Generation', unit = 'route'):
    
    route = gpd.read_file(os.path.join(GRADIENT_DATA_PATH,file))
    route.crs = 'epsg:32614'
    
    for month_bucket in range(4):
        for hour_bucket in range(3):
            
            df = speed_dfs[month_bucket][hour_bucket]
            
            bucket = "%d_%d" % (month_bucket, hour_bucket)
            
            out_path = os.path.join(OUTPUT_PATH, bucket, file.replace("shp","csv"))
            
            add_speed(df, route)
            
            to_cycle(route, out_path)

Cycle Generation:   0%|               | 0/109 [00:10<?, ?route/s]


IndexError: list index out of range